In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [2]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def prepare_data():
    """
    prepare the data
    Returns:
        x_train(ndarray): #training data (50000,32,32,3)
        x_test(ndarray) : #testing data  (10000,32,32,3) 
        y_train(ndarray): #training answer , one-hot encoded (50000,10)
        y_test(ndarray) : #testing data answer, one-hot encoded (10000,10)
        y_test_label (ndarray) : #
    """
    
    #load data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    #normalization
    (x_train, x_test) = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    
    #one-hot encoding
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    
    return x_train, x_test, y_train, y_test

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    """
    building model
    """
    
    #build dequential object
    model = Sequential()
    
    #conv layer 1
    model.add(Conv2D(filters = 64,
                     kernel_size = 3,
                     padding = 'same',
                     activation = 'relu',
                     input_shape = (32,32,3)))
    
    #2x2 pooling
    model.add(MaxPooling2D(pool_size = 2))
    
    #Conv layer 2
    model.add(Conv2D(filters = 128,
                     kernel_size = 3,
                     padding = 'same',
                     activation = 'relu'))
    
    #2x2 pooling
    model.add(MaxPooling2D(pool_size = 2))
    
    #Conv layer 3
    model.add(Conv2D(filters = 256,
                     kernel_size = 3,
                     padding = 'same',
                     activation = 'relu'))
    
    #2x2 pooling
    model.add(MaxPooling2D(pool_size=2))
    
    #flatten
    model.add(Flatten())
    
    #Dropout
    model.add(Dropout(0.4))
    
    #7th layer, full connect
    model.add(Dense(512, 
                    activation = 'relu'))
    
    #output:
    model.add(Dense(10,
                    activation = 'softmax'))
    
    #compile, adam as optimizer
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizers.Adam(lr = 0.001),
                  metrics = ['accuracy'])
    
    return model

In [4]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import Callback

class LRHistory(Callback):
    def on_train_begin(self, logs = {}):
        self.acc = []
        self.lr = []
    def on_epoche_end(self, batch, logs = {}):
        self.acc.append(logs.get('acc'))
        self.lr.append(step_decay(len(self.acc)))
        
def step_decay(epoch):
    """
    use step decay
    return learning rate
    """
    
    initial_lrate = 0.001 #initial lrate
    drop = 0.5            #decay rate
    epochs_drop = 10.0   #decay after 10 times
    lrate = initial_lrate * math.pow(drop, # "drop" to the power of lrate
                                     math.floor((epoch)/epochs_drop)) 
    return lrate
    
def train(x_train, x_test, y_train, y_test):
    
    model = make_convlayer()
    lr_history = LRHistory()
    lrate = LearningRateScheduler(step_decay)
    callbacks_list = [lr_history, lrate]
    
    #data augmentation
    datagen = ImageDataGenerator(width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 rotation_range = 10,
                                 zoom_range = 0.1,
                                 horizontal_flip = True)
    #batch size
    batch_size = 128
    
    #training times
    epochs = 100
    
    #train
    step_epoch = x_train.shape[0] // batch_size
    history = model.fit(datagen.flow(x_train,
                                     y_train,
                                     batch_size = batch_size),
                        steps_per_epoch = step_epoch, #steps in each batch
                        epochs = epochs,              #training times
                        verbose = 1,                  #output
                        validation_data = (x_test, y_test),
                        callbacks = callbacks_list)
    return history, lr_history
    

In [5]:
x_train, x_test, y_train, y_test = prepare_data()


170508288/170498071 [==============================] - 3s 0us/step


In [6]:
%%time
history, lr_history = train(x_train, x_test, y_train, y_test)

2022-06-10 06:05:18.174561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 06:05:18.277087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 06:05:18.277872: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-10 06:05:18.279252: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/100


2022-06-10 06:05:22.992420: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


390/390 [==============================] - 36s 74ms/step - loss: 1.6144 - accuracy: 0.4080 - val_loss: 1.2188 - val_accuracy: 0.5595
Epoch 2/100
390/390 [==============================] - 28s 72ms/step - loss: 1.2322 - accuracy: 0.5606 - val_loss: 1.0148 - val_accuracy: 0.6364
Epoch 3/100
390/390 [==============================] - 29s 73ms/step - loss: 1.0802 - accuracy: 0.6171 - val_loss: 0.8855 - val_accuracy: 0.6922
Epoch 4/100
390/390 [==============================] - 28s 72ms/step - loss: 0.9716 - accuracy: 0.6562 - val_loss: 0.8197 - val_accuracy: 0.7105
Epoch 5/100
390/390 [==============================] - 28s 73ms/step - loss: 0.9023 - accuracy: 0.6820 - val_loss: 0.7826 - val_accuracy: 0.7242
Epoch 6/100
390/390 [==============================] - 29s 73ms/step - loss: 0.8531 - accuracy: 0.7012 - val_loss: 0.7273 - val_accuracy: 0.7478
Epoch 7/100
390/390 [==============================] - 28s 72ms/step - loss: 0.8080 - accuracy: 0.7165 - val_loss: 0.6983 - val_accuracy: 0.75

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

x_range = list(range(1,len(lr_history.lr)+1))

#set printing area
plt.figure(figsize(15,10))            #adjust the size of the figures
plt.subplots(adjust(wspace=0.2))      #print on the 2x1 gridlines
plt.subplot(2,1,1)                    #plot accuray and validatlr_historyion accuracy
plt.plot(x_range, history['accuary'],
         label = 'train', linestyle = '--')
plt.plot(x_range, history.history['val_accuracy'],
         label = 'Val Acc')
plt.legend()                          #show legend
plt.grid()                            #show grid line
plt.xlabel('Epoch')                   #show x axis label
plt.ylabel('Acc')                     #show y axis label

#plt under 2x1 gridline
plt.subplot(2,1,2)                    #plot learning rate
plt.plot(x_range, lr_history.lr, lebel = 'Learning Rate')
plt.legend()                          #plot legend
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')

plt.show()



NameError: name 'figsize' is not defined